You need to install beautifulsoup4, requests, urllib 
- "pip install beautifulsoup4"
- "pip install requests"
- "pip install urllib"

In [295]:
from bs4 import BeautifulSoup
import urllib.request,sys,time
from urllib.request import Request, urlopen
import requests
import pandas as pd
import csv

In [296]:
def get_html(url):
    try:
        req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
        webpage = urlopen(req).read()
        page_soup = BeautifulSoup(webpage, "html.parser")
    except:
        print("Error with parsing:", url)
    
    return page_soup

In [297]:
def get_pages(url):
    page_links = []
    for i in range(1, 811):
        page_links.append(url + 'page/' + str(i) + '/')
    return page_links

In [298]:
def get_article_page_links(url):
    links = set()
    try:
        page_html = get_html(url)
        articles = page_html.findAll("article")
    except:
        print("Error with parsing:", url)
        return []
        for article in articles:
            try:
                for link in article.select("a"):
                    links.add(link['href'])
            except:
                continue
    return links

In [299]:
def get_true_false_info(url):
    false_phrase = ''
    true_phrase = ''
    count = 1
    
    try:
        page_soup = get_html(url)
        false = page_soup.findAll("h1", {"class": "entry-title"})
        true = page_soup.findAll("p", {"class": "entry-content__text entry-content__text--explanation"})
    except:
        print("Error with url:", url)
    
    try:
        for phrase in false[0]:
            if count == 3:
                false_phrase = phrase
            count += 1
    except:
        print('Error with false phrase')
        return []
    
    try:
        count = 1
        for phrase in true[0]:
            if count == 1:
                true_phrase = phrase
            count += 1
    except:
        print('Error with true phrase')
        return []
    
    return [false_phrase[1:], true_phrase[13:]]

In [300]:
def get_statements(url):
    pages = get_pages(url)
    article_page_links = set()
    page_count = 1
    for page in pages:
        print("This is page", page_count)
        article_page_links.update(get_article_page_links(page))
        page_count += 1
    true_false = dict()
    link_count = 1
    for link in article_page_links:
        print("This is link", link_count)
        true_false[link] = get_true_false_info(link)
        link_count += 1
    return true_false

In [301]:
def get_false_statements(true_false):
    false = []
    for statements in true_false.values():
        false.append(statements[0])
    return false

In [302]:
def get_true_statements(true_false):
    true = []
    for statements in true_false.values():
        true.append(statements[1])
    return true

In [303]:
def save_csv(save_dict, name):
    with open(name, 'w') as f:
        f.write("Link; False Statement; True Statement\n")
        for key in save_dict.keys():
            f.write("%s; %s; %s\n" % (key, save_dict[key][0], save_dict[key][1]))

In [ ]:
if __name__ == "__main__":
    home_page_url = 'https://www.poynter.org/ifcn-covid-19-misinformation/'
    statement_dict = get_statements(home_page_url)
    save_csv(statement_dict, "false_news_data.csv")

This is page 1
This is page 2
This is page 3
This is page 4
This is page 5
This is page 6
This is page 7
This is page 8
This is page 9
This is page 10
This is page 11
This is page 12
This is page 13
This is page 14
This is page 15
This is page 16
This is page 17
This is page 18
This is page 19
This is page 20
This is page 21
This is page 22
This is page 23
This is page 24
This is page 25
This is page 26
This is page 27
This is page 28
This is page 29
This is page 30
This is page 31
This is page 32
This is page 33
This is page 34
This is page 35
This is page 36
This is page 37
This is page 38
This is page 39
This is page 40
This is page 41
This is page 42
This is page 43
This is page 44
This is page 45
This is page 46
This is page 47
This is page 48
This is page 49
This is page 50
This is page 51
This is page 52
This is page 53
This is page 54
This is page 55
This is page 56
This is page 57
This is page 58
This is page 59
This is page 60
This is page 61
This is page 62
This is page 63
T